# Capstone Project: Battle of Neighborhoods - Capstone Project - UDevi


In [1]:
# importing old libraries
from bs4 import BeautifulSoup #Python Web scraping library
import requests # library to handle requests
import json # library to handle JSON files
import xml
import pandas as pd #Python library data manipulation and analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
# importing new libraries
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Collecting Toronto Neighborhoods and their Postal codes by web scraping

In [314]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(url).text

soup = BeautifulSoup(raw_wikipedia_page,'lxml')
#print(soup.prettify())               

In [315]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighborhood  = []

# print(table)

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighborhood_var  = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighborhood_var = str(td_cell.text).strip()
            tag_a_Neighborhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighborhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighborhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighborhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighborhood.append(Neighborhood_var)

In [316]:
unique_p = set(Postcode)
# print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighborhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighborhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighborhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighborhood_u.append(n_var)         

In [318]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighborhood':Neighborhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
#df_toronto.to_csv('toronto_part1.csv')
df_toronto.head()

#df_toronto

,Postcode,Borough,Neighborhood
0,M9A,Etobicoke,Islington Avenue
1,M4H,East York,Thorncliffe Park
2,M1B,Scarborough,"Rouge, Malvern"
3,M9W,Etobicoke,Northwest
4,M9L,North York,Humber Summit


### Loading the Toronto Neighborhood coordinates data from Geospatial_Coordinates.csv file¶

In [319]:
df_coords = pd.read_csv('Geospatial_Coordinates.csv')

In [320]:
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [321]:
df_toronto = pd.merge(df_toronto, df_coords.rename(columns={'Postal Code':'Postcode'}), on='Postcode',  how='left')

In [324]:
df_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
1,M4H,East York,Thorncliffe Park,43.705369,-79.349372
2,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
3,M9W,Etobicoke,Northwest,43.706748,-79.594054
4,M9L,North York,Humber Summit,43.756303,-79.565963


Create a map of North York and its neighborhoods

In [325]:
address = 'North York,Toronto'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The geograpical coordinate of North York ,Toronto are {}, {}.'.format(latitude_x, longitude_y))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of North York ,Toronto are 43.7709163, -79.4124102.


In [326]:
borough_data = df_toronto[df_toronto['Borough'] == 'North York'].reset_index(drop=True)
borough_data.head(7)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M9L,North York,Humber Summit,43.756303,-79.565963
1,M3J,North York,"Northwood Park, York University",43.767980,-79.487262
2,M2H,North York,Hillcrest Village,43.803762,-79.363452
3,M2J,North York,Henry Farm,43.778517,-79.346556
4,M6L,North York,Maple Leaf Park,43.713756,-79.490074
5,M3H,North York,"Bathurst Manor, Wilson Heights",43.754328,-79.442259
6,M3K,North York,CFB Toronto,43.737473,-79.464763


In [327]:
# create map of North York,Toronto using latitude and longitude values
map_northyork = folium.Map(location=[latitude_x, longitude_y], zoom_start=12)

In [328]:
# add markers to map
#for lat, lng, nei in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
for lat, lng, nei in zip(borough_data['Latitude'], borough_data['Longitude'], borough_data['Neighborhood']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

In [329]:
#neighborhood_1=input("Select a neighborhood from the main_df")
CLIENT_ID = 'HB4BJSNOUSOLGGTW2FE2S4T4GMBC1WY3H5THCRCQ4QCR3UGU' # Foursquare ID
CLIENT_SECRET = 'OSOOXGGSMMHJSU12PYPB2T1XGI1LHYNVW0KLSXG2FV33NP50' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [330]:
borough_data = df_toronto[df_toronto['Borough'] == 'North York'].reset_index(drop=True)
borough_data

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M9L,North York,Humber Summit,43.756303,-79.565963
1,M3J,North York,"Northwood Park, York University",43.767980,-79.487262
2,M2H,North York,Hillcrest Village,43.803762,-79.363452
3,M2J,North York,Henry Farm,43.778517,-79.346556
4,M6L,North York,Maple Leaf Park,43.713756,-79.490074
5,M3H,North York,"Bathurst Manor, Wilson Heights",43.754328,-79.442259
6,M3K,North York,CFB Toronto,43.737473,-79.464763
7,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
8,M2K,North York,Bayview Village,43.786947,-79.385975
9,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714


In [331]:
address = 'North York,Toronto'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_n1 = location.latitude
longitude_n1 = location.longitude
print('The geograpical coordinate of North York,Toronto are {}, {}.'.format(latitude_x, longitude_y))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of North York,Toronto are 43.7709163, -79.4124102.


In [332]:
radius = 700 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_n1, 
   longitude_n1, 
    radius, 
   LIMIT)

In [333]:
results = requests.get(url).json()

In [334]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.events.count', 'venue.events.summary', 'venue.id',
       'venue.location.address', 'venue.location.cc', 'venue.location.city',
       'venue.location.country', 'venue.location.crossStreet',
       'venue.location.distance', 'venue.location.formattedAddress',
       'venue.location.labeledLatLngs', 'venue.location.lat',
       'venue.location.lng', 'venue.location.neighborhood',
       'venue.location.postalCode', 'venue.location.state', 'venue.name',
       'venue.photos.count', 'venue.photos.groups', 'venue.venuePage.id'],
      dtype='object')

In [335]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [336]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,The Captain's Boil,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",43.773255,-79.413805
1,Loblaws,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",43.768648,-79.412597
2,Dakgogi,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",43.773010,-79.413875
3,Burrito Boyz,"[{'id': '4bf58dd8d48988d153941735', 'name': 'B...",43.773054,-79.414082
4,Aroma Espresso Bar,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",43.769449,-79.413081
5,The Keg,"[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...",43.766579,-79.412131
6,Starbucks,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.768192,-79.413021
7,Cineplex Cinemas Empress Walk,"[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...",43.768625,-79.412613
8,Konjiki Ramen,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",43.766998,-79.412222
9,Sushi Bong,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",43.775428,-79.413654


In [337]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,The Captain's Boil,Seafood Restaurant,43.773255,-79.413805
1,Loblaws,Grocery Store,43.768648,-79.412597
2,Dakgogi,Korean Restaurant,43.773010,-79.413875
3,Burrito Boyz,Burrito Place,43.773054,-79.414082
4,Aroma Espresso Bar,Café,43.769449,-79.413081
5,The Keg,Steakhouse,43.766579,-79.412131
6,Starbucks,Coffee Shop,43.768192,-79.413021
7,Cineplex Cinemas Empress Walk,Movie Theater,43.768625,-79.412613
8,Konjiki Ramen,Ramen Restaurant,43.766998,-79.412222
9,Sushi Bong,Sushi Restaurant,43.775428,-79.413654


In [359]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [360]:
ny_venues = getNearbyVenues(names=borough_data['Neighborhood'],
                                   latitudes=borough_data['Latitude'],
                                   longitudes=borough_data['Longitude']
                                  )

Humber Summit
Northwood Park, York University
Hillcrest Village
Henry Farm
Maple Leaf Park
Bathurst Manor, Wilson Heights
CFB Toronto
Lawrence Heights, Lawrence Manor
Bayview Village
Silver Hills, York Mills
Emery, Humberlea
Downsview West
Willowdale West
Parkwoods
Flemingdon Park
Victoria Village
Bedford Park
Newtonbrook, Willowdale
Glencairn


In [361]:
print(ny_venues.shape)

(265, 7)


In [362]:
ny_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights",19,19,19,19,19,19
Bayview Village,10,10,10,10,10,10
Bedford Park,30,30,30,30,30,30
CFB Toronto,4,4,4,4,4,4
Downsview West,8,8,8,8,8,8


In [363]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 113 uniques categories.


In [364]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]
ny_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bridal Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Electronics Store,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Hardware Store,Hockey Arena,Home Service,Housing Development,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Lingerie Store,Liquor Store,Massage Studio,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Nightclub,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Portuguese Restaurant,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoothie Shop,Sporting Goods Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Humber Summit,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Humber Summit,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Humber Summit,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Humber Summit,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Northwood Park, York University",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [365]:
ny_onehot.shape

(265, 114)

In [366]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bridal Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Electronics Store,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Hardware Store,Hockey Arena,Home Service,Housing Development,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Lingerie Store,Liquor Store,Massage Studio,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Nightclub,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Portuguese Restaurant,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoothie Shop,Sporting Goods Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor, Wilson Heights",0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.052632,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.105263,0.000000,0.052632,0.0,0.0,0.000000,0.0,0.052632,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.052632,0.052632,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.052632,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.052632,0.0,0.0,0.000,0.0,0.0,0.052632,0.0,0.052632,0.052632,0.0,0.0,0.000000,0.0,0.052632,0.0,0.052632,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.052632,0.0,0.000000,0.0,0.0,0.0,0.0,0.052632,0.000,0.0,0.0
1,Bayview Village,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.100000,0.0,0.0,0.0,0.1,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.100000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.1,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,0.0,0.100000,0.000000,0.0,0.0,0.000000,0.0,0.100000,0.0,0.000000,0.0,0.0,0.000000,0.1,0.1,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.0
2,Bedford Park,0.0,0.00,0.033333,0.0,0.0,0.0,0.033333,0.033333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.000000,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.100000,0.033333,0.000000,0.0,0.0,0.033333,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.033333,0.066667,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.033333,0.033333,0.0,0.033333,0.0,0.0,0.0,0.000000,0.033333,0.0,0.1,0.0,0.033333,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,0.0,0.033333,0.033333,0.0,0.0,0.033333,0.0,0.033333,0.0,0.033333,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.033333,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.0
3,CFB Toronto,0.0,0.25,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.250000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.

In [367]:
ny_grouped.shape

(19, 114)

In [368]:
num_top_venues = 5
for hood in ny_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights----
           venue  freq
0    Coffee Shop  0.11
1  Shopping Mall  0.05
2     Restaurant  0.05
3          Diner  0.05
4    Bridal Shop  0.05


----Bayview Village----
                venue  freq
0                Bank   0.2
1  Chinese Restaurant   0.1
2            Pharmacy   0.1
3                Café   0.1
4          Restaurant   0.1


----Bedford Park----
                  venue  freq
0           Coffee Shop  0.10
1    Italian Restaurant  0.10
2  Fast Food Restaurant  0.07
3        Cosmetics Shop  0.03
4                  Café  0.03


----CFB Toronto----
                       venue  freq
0                    Airport  0.25
1                Coffee Shop  0.25
2             Sandwich Place  0.25
3                       Park  0.25
4  Latin American Restaurant  0.00


----Downsview West----
           venue  freq
0           Park  0.12
1    Pizza Place  0.12
2    Coffee Shop  0.12
3           Bank  0.12
4  Moving Target  0.12


----Emery, Humberlea----
    

In [369]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [370]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights",Coffee Shop,Sandwich Place,Bridal Shop,Restaurant,Pizza Place,Pharmacy,Deli / Bodega,Shopping Mall,Park,Community Center
1,Bayview Village,Bank,Grocery Store,Restaurant,Japanese Restaurant,Skate Park,Skating Rink,Chinese Restaurant,Pharmacy,Café,Community Center
2,Bedford Park,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Butcher,Pub,Pizza Place,Pharmacy,Café,Liquor Store,Juice Bar
3,CFB Toronto,Park,Airport,Coffee Shop,Sandwich Place,Dessert Shop,Chinese Restaurant,Clothing Store,Comfort Food Restaurant,Community Center,Construction & Landscaping
4,Downsview West,Moving Target,Coffee Shop,Shopping Mall,Bank,Grocery Store,Pizza Place,Park,Vietnamese Restaurant,Clothing Store,Chinese Restaurant


Run k-means to cluster the neighborhoods into 4 clusters

In [371]:
# This list is created manually 
important_list_of_features = [
 'Neighborhood',
#'Neighborhood Latitude',
# 'Neighborhood Longitude',
#    'Latitude',
#    'location.latitude',
'American Restaurant',
'Asian Restaurant',
'Caribbean Restaurant',
'Chinese Restaurant',
'Comfort Food Restaurant',
'Dim Sum Restaurant',
'Empanada Restaurant',
'Fast Food Restaurant',
'Greek Restaurant',
'Indian Restaurant',
'Italian Restaurant',
'Japanese Restaurant',
'Latin American Restaurant',
'Middle Eastern Restaurant',
'Portuguese Restaurant',
'Restaurant',
'Seafood Restaurant',
'Sushi Restaurant',
'Thai Restaurant',
'Vietnamese Restaurant',
]

In [373]:
ny_onehot = ny_onehot[important_list_of_features].groupby('Neighborhood').sum()

In [374]:
ny_onehot.head()

,American Restaurant,Asian Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Dim Sum Restaurant,Empanada Restaurant,Fast Food Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Middle Eastern Restaurant,Portuguese Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Thai Restaurant,Vietnamese Restaurant
Neighborhood,,,,,,,,,,,,,,,,,,,,
"Bathurst Manor, Wilson Heights",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0
Bayview Village,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
Bedford Park,1,0,0,0,1,0,0,2,1,1,3,0,0,0,0,1,0,1,1,0
CFB Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Downsview West,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [376]:
feat_name_list = list(ny_onehot.columns)
restaurant_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
ny_onehot['Total Restaurants'] = ny_onehot[restaurant_list].sum(axis = 1)
ny_onehot = ny_onehot.drop(columns = restaurant_list)


feat_name_list = list(ny_onehot.columns)
#joint_list = []


#for counter, value in enumerate(feat_name_list):
#    if value.find('Joint') != (-1):
#        joint_list.append(value)
        
#ny_onehot['Total Joints'] = ny_onehot[joint_list].sum(axis = 1)
#ny_onehot = ny_onehot.drop(columns = joint_list)


In [377]:
ny_onehot

,Total Restaurants
Neighborhood,
"Bathurst Manor, Wilson Heights",3
Bayview Village,3
Bedford Park,12
CFB Toronto,0
Downsview West,1
"Emery, Humberlea",0
Flemingdon Park,9
Glencairn,4
Henry Farm,14


In [378]:
ny_onehot.sort_values(axis = 0, by = ['Total Restaurants'], ascending=False)

,Total Restaurants
Neighborhood,
Henry Farm,14
Bedford Park,12
Flemingdon Park,9
"Northwood Park, York University",4
"Lawrence Heights, Lawrence Manor",4
Glencairn,4
"Bathurst Manor, Wilson Heights",3
Bayview Village,3
Hillcrest Village,2


# Result:
Best Neighborhood is Henry Farm with 14 restaurants;  
Second best Neighborhood is Bedford Park with 12 restaurants  
Third best Neighborhood is Flemingdon Park with 9 restaurants 
